# G2Audit Guide - Summary Data
The getSummaryData() and getSummaryDataDirect() methods obtain general summary data about your current audit session.

## Prepare Environment

In [ ]:
import com.senzing.g2.engine.G2Audit;
import com.senzing.g2.engine.G2AuditJNI;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
String str;
public static void RenderJSON(Object obj){
    str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

## Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
RenderJSON(senzingConfigJson);

## G2Audit
The G2Audit API allows you to pull data reporting data back on how one data source is related to another. Once you are done loading one or more data sources, use this API to query information that can be useful for reporting, browsing, or auditing results.

### Initialization

To start using Senzing G2Audit, create and initialize an instance.
This should be done once per process.
The `initV2()` method accepts the following parameters:

- **moduleName:** A short name given to this instance of the G2Audit object.
- **senzingConfigJson:** A JSON string containing configuration parameters.
- **verboseLogging:** A boolean which enables diagnostic logging.

Calling this function will return "0" upon success.

In [ ]:
String moduleName = "ExampleG2Audit";
boolean verboseLogging = true;

G2Audit g2audit = new G2AuditJNI();
int return_code = g2audit.initV2(moduleName, senzingConfigJson, verboseLogging);

if(return_code!=0)
    System.out.print(g2audit.getLastException());
else
    System.out.print(return_code);

### openSession()
Create a new session.

In [ ]:
long g2audit_session = g2audit.openSession();

### getSummaryData()

Call G2Audit's `getSummaryData()` method and pretty-print results.

The `getSummaryData()` method accepts the following parameters:

- **g2audit_session:** A long integer representing a previously-created G2Audit object.
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2audit.getSummaryData(g2audit_session, response);

if(return_code!=0)
    System.out.print(g2audit.getLastException());
else
    RenderJSON(response);

### closeSession()

In [ ]:
g2audit.closeSession(g2audit_session);

### getSummaryDataDirect()

Call G2Audit's `getSummaryDataDirect()` method and pretty-print results.

The `getSummaryDataDirect()` method accepts the following parameters:

- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2audit.getSummaryDataDirect(response);

if(return_code!=0)
    System.out.print(g2audit.getLastException());
else
    RenderJSON(response);